In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import json
import numpy as np
import math
from nltk import word_tokenize
import torchtext

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [27]:
train_data = []
train_inp_file = 'processed_data/GCDC/Clinton_train.jsonl'
with open(train_inp_file, 'r') as f:
    for line in f:
        json_obj = json.loads(line)
        document = []
        for sentence in json_obj['sentences']:
            # document.append([word.lower() for word in sentence])
            for word in sentence:
                document.append(word.lower())
            # add <EOS> at the end of each sentence
            document.append('<EOS>')
        label = json_obj['label']
        
        train_data.append((document, label))
   

test_data = []
test_inp_file = 'processed_data/GCDC/Clinton_test.jsonl'
with open(test_inp_file, 'r') as f:
    for line in f:
        json_obj = json.loads(line)
        document = []
        for sentence in json_obj['sentences']:
            # document.append([word.lower() for word in sentence])
            for word in sentence:
                document.append(word.lower())
            # add <EOS> at the end of each sentence
            document.append('<EOS>')
        label = json_obj['label']
        test_data.append((document, label))
        
        
print(len(train_data))
print(train_data[0])
print(train_data[1])
print(train_data[2])
print(len(test_data))
print(test_data[0])



800
(['t', 'w', 'o', 'o', 'p', 't', 'i', 'o', 'n', 's', 't', 'h', 'e', 'u', 's', 'v', 'i', 'e', 'w', 's', 't', 'h', 'e', 't', 'r', 'a', 'n', 's', 'i', 't', 'i', 'o', 'n', 'a', 'l', 'n', 'a', 't', 'i', 'o', 'n', 'a', 'l', 'c', 'o', 'u', 'n', 'c', 'i', 'l', 'a', 's', 't', 'h', 'e', 's', 'o', 'l', 'e', '/', 'o', 'n', 'l', 'y', 'l', 'e', 'g', 'i', 't', 'i', 'm', 'a', 't', 'e', 'i', 'n', 't', 'e', 'r', 'l', 'o', 'c', 'u', 't', 'o', 'r', 'o', 'f', 't', 'h', 'e', 'l', 'i', 'b', 'y', 'a', 'n', 'p', 'e', 'o', 'p', 'l', 'e', 'd', 'u', 'r', 'i', 'n', 'g', 't', 'h', 'i', 's', 'i', 'n', 't', 'e', 'r', 'i', 'm', 'p', 'e', 'r', 'i', 'o', 'd', ',', 'a', 's', 'l', 'i', 'b', 'y', 'a', 'n', 's', 'c', 'o', 'm', 'e', 't', 'o', 'g', 'e', 't', 'h', 'e', 'r', 't', 'o', 'p', 'l', 'a', 'n', 't', 'h', 'e', 'i', 'r', 'o', 'w', 'n', 'f', 'u', 't', 'u', 'r', 'e', 'a', 'n', 'd', 'a', 'p', 'e', 'r', 'm', 'a', 'n', 'e', 'n', 't', ',', 'i', 'n', 'c', 'l', 'u', 's', 'i', 'v', 'e', 'c', 'o', 'n', 's', 't', 'i', 't', 'u',

In [15]:
# pad the document to the same length
def pad_document(document, max_len):
    if len(document) >= max_len:
        return document[:max_len]
    else:
        return document + ['<PAD>'] * (max_len - len(document))

# find the max length of the document
max_len = 0
for document, label in train_data:
    max_len = max(max_len, len(document))
for document, label in test_data:
    max_len = max(max_len, len(document))
print(max_len)

# pad the document
for i in range(len(train_data)):
    train_data[i] = (pad_document(train_data[i][0], max_len), train_data[i][1])
for i in range(len(test_data)):
    test_data[i] = (pad_document(test_data[i][0], max_len), test_data[i][1])
print(train_data[0])
print(test_data[0])

415
(['two', 'options', 'the', 'us', 'views', 'the', 'transitional', 'national', 'council', 'as', 'the', 'sole', '/', 'only', 'legitimate', 'interlocutor', 'of', 'the', 'libyan', 'people', 'during', 'this', 'interim', 'period', ',', 'as', 'libyans', 'come', 'together', 'to', 'plan', 'their', 'own', 'future', 'and', 'a', 'permanent', ',', 'inclusive', 'constitutional', 'system', 'that', 'protects', 'the', 'rights', 'of', 'all', 'libyans', '.', '<EOS>', 'this', 'is', 'in', 'contrast', 'to', 'the', 'qadhafi', 'regime', ',', 'which', 'has', 'lost', 'all', 'legitimacy', 'to', 'rule', '.', '<EOS>', 'the', 'us', 'views', 'the', 'transitional', 'national', 'council', 'as', 'the', 'legitimate', 'interlocutor', 'of', 'the', 'libyan', 'people', 'during', 'this', 'interim', 'period', ',', 'as', 'libyans', 'come', 'together', 'to', 'plan', 'their', 'own', 'future', 'and', 'a', 'permanent', ',', 'inclusive', 'constitutional', 'system', 'that', 'protects', 'the', 'rights', 'of', 'all', 'libyans', '.'

In [16]:
# # build vocabulary using training data
# vocabulary = torchtext.vocab.build_vocab_from_iterator([document for document, label in train_data], specials=['<UNK>', '<PAD>', '<EOS>'], min_freq=2)
# vocabulary.set_default_index(vocabulary['<UNK>'])

# print(len(vocabulary))
# print(vocabulary.get_itos()[:10])



In [17]:
# # convert words to indices
# train_indices = []
# for document, label in train_data:
#     document_indices = []
#     for word in document:
#         document_indices.append(vocabulary[word])
#     train_indices.append((document_indices, label))

# test_indices = []
# for document, label in test_data:
#     document_indices = []
#     for word in document:
#         document_indices.append(vocabulary[word])
#     test_indices.append((document_indices, label))


# # subtract 1 from the label
# for i in range(len(train_indices)):
#     train_indices[i] = (train_indices[i][0], train_indices[i][1] - 1)
# for i in range(len(test_indices)):
#     test_indices[i] = (test_indices[i][0], test_indices[i][1] - 1)

# print(train_indices[0])
# print(test_indices[0])

In [18]:
# # create dataloaders
# batch_size = 4
# # convert to tensors
# train_indices = [(torch.tensor(document_indices), label) for document_indices, label in train_indices]
# test_indices = [(torch.tensor(document_indices), label) for document_indices, label in test_indices]
# # create dataloaders
# train_loader = DataLoader(train_indices, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_indices, batch_size=batch_size, shuffle=False)

# print(len(train_loader))

In [19]:
# use pre-trained transformer model BERT to classify documents into three classes
# BERT is a transformer model pre-trained on large corpus

from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')

print(tokenizer.vocab_size)
print(bert.config.hidden_size)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


30522
768


In [22]:
# use the bert tokenizer to tokenize the document and convert the tokens to indices
train_indices = []
for document, label in train_data:
    document_indices = []
    for word in document:
        document_indices.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(word)))
    train_indices.append((document_indices, label-1))

test_indices = []
for document, label in test_data:
    document_indices = []
    for word in document:
        document_indices.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(word)))
    test_indices.append((document_indices, label-1))

print(train_indices[0])
print(test_indices[0])

([[2048], [7047], [1996], [2149], [5328], [1996], [17459], [2120], [2473], [2004], [1996], [7082], [1013], [2069], [11476], [6970, 4135, 12690, 2953], [1997], [1996], [19232], [2111], [2076], [2023], [9455], [2558], [1010], [2004], [19232, 2015], [2272], [2362], [2000], [2933], [2037], [2219], [2925], [1998], [1037], [4568], [1010], [18678], [6543], [2291], [2008], [18227], [1996], [2916], [1997], [2035], [19232, 2015], [1012], [1026, 1041, 2891, 1028], [2023], [2003], [1999], [5688], [2000], [1996], [1053, 4215, 3270, 8873], [6939], [1010], [2029], [2038], [2439], [2035], [22568], [2000], [3627], [1012], [1026, 1041, 2891, 1028], [1996], [2149], [5328], [1996], [17459], [2120], [2473], [2004], [1996], [11476], [6970, 4135, 12690, 2953], [1997], [1996], [19232], [2111], [2076], [2023], [9455], [2558], [1010], [2004], [19232, 2015], [2272], [2362], [2000], [2933], [2037], [2219], [2925], [1998], [1037], [4568], [1010], [18678], [6543], [2291], [2008], [18227], [1996], [2916], [1997], [2

In [ ]:
# define a classifier
class Classifier(nn.Module):
    def __init__(self, hidden_size, num_classes):
        super(Classifier, self).__init__()
        self.linear = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        return self.linear(x)
    
# define a transformer model
class Transformer(nn.Module):
    def __init__(self, hidden_size, num_classes):
        super(Transformer, self).__init__()
        self.bert = bert
        self.classifier = Classifier(hidden_size, num_classes)
        
    def forward(self, x):
        # x: (batch_size, seq_len)
        # x: (batch_size, seq_len, hidden_size)
        x = self.bert(x)[0]
        # x: (batch_size, hidden_size)
        x = x[:, 0, :]
        # x: (batch_size, num_classes)
        x = self.classifier(x)
        return x

In [ ]:
# define a function to train the model
def train(model, optimizer, criterion, train_loader, test_loader, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        for document, label in train_loader:
            # document: (batch_size, seq_len)
            # label: (batch_size)
            document = document.to(device)
            label = label.to(device)
            # output: (batch_size, num_classes)
            output = model(document)
            # output: (batch_size, num_classes)
            loss = criterion(output, label)
            # backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print('Epoch: {}, Loss: {}'.format(epoch, loss.item()))
        test(model, test_loader)

# define a function to test the model
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for document, label in test_loader:
            # document: (batch_size, seq_len)
            # label: (batch_size)
            document = document.to(device)
            label = label.to(device)
            # output: (batch_size, num_classes)
            output = model(document)
            # output: (batch_size)
            output = torch.argmax(output, dim=1)
            correct += torch.sum(torch.eq(output, label)).item()
            total += len(label)
    print('Accuracy: {}'.format(correct / total))

# define a function to predict the label of a document
def predict(model, document):
    model.eval()
    with torch.no_grad():
        # document: (seq_len)
        document = torch.LongTensor(document).unsqueeze(0).to(device)
        # output: (1, num_classes)
        output = model(document)
        # output: (1)
        output = torch.argmax(output, dim=1)
        return output.item()


In [ ]:
# create a transformer model
hidden_size = bert.config.hidden_size
num_classes = 3
model = Transformer(hidden_size, num_classes).to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# train the model
num_epochs = 10
train(model, optimizer, criterion, train_loader, test_loader, num_epochs)

Epoch: 0, Loss: 0.6066164970397949
Accuracy: 0.555
Epoch: 1, Loss: 0.7545852065086365


KeyboardInterrupt: 

In [25]:
sample = 'I love this movie!'
tokenss = tokenizer.tokenize(sample)
print(tokenss)
indices = tokenizer.convert_tokens_to_ids(tokenss)
print(indices)

['i', 'love', 'this', 'movie', '!']
[1045, 2293, 2023, 3185, 999]
